In [ ]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, BoxSelectTool, LassoSelectTool, PanTool, WheelZoomTool, ResetTool, UndoTool, RedoTool, SaveTool, Circle
from bokeh.layouts import gridplot
from bokeh.tile_providers import get_provider

### Reading in data ###

df = pd.read_csv('estuary_cleaned.csv')
df = df.drop_duplicates(subset='Site_Code') #We are only looking at statistics general to each site, so here we are removing rows with duplicate site codes

### Converting latitude and longitude to Web Mercator Projection for map, adapted from https://stackoverflow.com/questions/57178783/how-to-plot-latitude-and-longitude-in-bokeh ###

k = 6378137
df["Longitude_merc"] = df["Longitude"] * (k * np.pi/180.0)
df["Latitude_merc"] = np.log(np.tan((90 + df["Latitude"]) * np.pi/360.0)) * k

### Setting up plot settings ###

output_file("project4_ADV_W15A_Group_4.html")
tooltips = [ #Adding hover information
    ("Estuary name", "@Estuary_name"),
    ("Estuary type", "@Estuary_type"),
    ("Longitude", "@Longitude"),
    ("Latitude", "@Latitude")
]
tools = [ #Choosing tools for toolbar
    HoverTool(tooltips=tooltips),
    PanTool(),
    WheelZoomTool(),
    BoxSelectTool(),
    LassoSelectTool(),
    UndoTool(),
    RedoTool(),
    ResetTool(),
    SaveTool()
]
source = ColumnDataSource(data=df)

### Scatter plot (left) ###

leftFig = figure(
    title="Percent cleared and percent urbanised",
    tools=tools,
    plot_height=600,
    plot_width=600,
    x_range=(0,100),
    y_range=(0,100),
    x_axis_label="Percentage of catchment cleared",
    y_axis_label="Percentage of catchment urbanised"
)
leftFig_points = leftFig.circle(
    "percent_cleared", "percent_urban",
    source=source,
    fill_color="darkcyan",
    size=10,
    alpha=0.5
)
leftFig_points.selection_glyph = Circle(
    fill_color="orangered",
    line_color=None,
    fill_alpha=0.5
)
leftFig_points.nonselection_glyph = Circle(
    fill_color="powderblue",
    line_color=None, 
    fill_alpha=0.3
)

### Map scatter plot (right) ###

rightFig = figure(
    title = "Location of site",
    tools=tools,
    plot_height=600,
    plot_width=600,
    x_range=(16200000,17600000),
    y_range=(-4600000,-3200000),
    x_axis_label="Longitude",
    y_axis_label="Latitude",
    x_axis_type="mercator", #This means the coordinates shown on the axes are in longitude and latitude rather than Web Mercator projection
    y_axis_type="mercator"
)
rightFig.add_tile(get_provider('CARTODBPOSITRON')) #This adds the map
rightFig_points = rightFig.circle(
    "Longitude_merc", "Latitude_merc",
    source=source,
    fill_color="darkcyan",
    size=10,
    alpha=0.5
)
rightFig_points.selection_glyph = Circle(
    fill_color="orangered",
    line_color=None,
    fill_alpha=0.5
)
rightFig_points.nonselection_glyph = Circle(
    fill_color="powderblue",
    line_color=None,
    fill_alpha = 0.3
)

### Final visualisation ###

p = gridplot([[leftFig, rightFig]])
show(p)